# RCOT on SUTs (c<i)

This notebook starts from an exemplary supply-use table (SUT) that features fewer commodities than industries (c<i). The cases of [c=i](./SU-RCOT_c=i.ipynb) and [c>i](./SU-RCOT_c+i-.ipynb) are illustrated in separate notebooks. All notebooks share the same basic structure. That is, an illustrative SUT is imported from a spreadsheet; solved for the cases of factor constraints being absent or present; and solved when new technologies are introduced.

<a id='toc'></a>

The outline of the current notebook is as follows:


0. [Data import and set-up](#data-import)
1. [Empirical system in coefficent form w/out factor constraints](#emp_0f)\
    1.1 [As-is solution by inversion (use of constructs)](#emp_0f_inv)\
    1.2 [As-is solution via linear program (of joint production system)](#emp_0f_lp)\
        1.2.1 [Relative (coefficient) formulation](#emp_0f_lp_rel)\
        1.2.2 [Absolute (transaction) formulation](#emp_0f_lp_abs)
2. [Empirical system with factor constraints](#emp_f_lp)
3. [Adding technology alternatives to the SUT system](#add_jp)\
    3.1 [An alternative for by-product production](#add_jp_1sp)\
    3.2 [An alternative for main-product production with secondary products](#add_jp_1jp)

<a id='data-import'></a>

## 0. Data import and set-up

In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using DataFrames

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c-i+.xlsx")

XLSXFile("../data/SUT_c-i+.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 6x4           A1:D6        
                  U+e 4x7           A1:G4        
                    F 5x6           A1:F5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D6"])
U = convert(Matrix{Float64}, xf["U+e!B2:F4"])
e = convert(Matrix{Float64}, xf["U+e!G2:G4"])
F = convert(Matrix{Float64}, xf["F!B2:F5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

As per the imported spreadsheets, $(V,U,e)$ are given in monetary units meaning that the commodity prices $p=i$. $F$ is in physical units and $\pi$ are the factor prices. As the system is rectangular (c<i), it cannot be solved for total output by inversion. Hence, either we solve via a linear program or we reallocate the by-products via constructs and thus allow for solution by inversion. For convenience, we pack the SUT system into an object that will be used in the following.

In [4]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
# For completeness, let's check the total factor costs (in this unconstrained system)
sut.pii'*sut.f

1×1 Matrix{Float64}:
 123.99999999999999

Back to [table of contents](#toc)
<a id='emp_0f'></a>

## 1. Empirical system w/out factor constraint
Since the given SUT-system is again rectangular, we cannot perform imputation or impact analyses with it directly by inversion. Instead, we solve it again through an LP and by inversion after having applied constructs.

Back to [table of contents](#toc)
<a id='emp_0f_inv'></a>

### 1.1 Solution by inversion via constructs
Various constructs exist to reallocate by-products. Not all of them work for rectangular SUT-systems, though. For completeness, we introduce the Models A (commodity technology construct, CTC), B (industry technology construct, ITC), C (fixed industry sales structure construct, FISC), and D (fixed product sales structure construct, FPSC) here, although only two of them will be appropriate in the present case of more commodities than industries. We compute the solution here via the ITC and compare it to the commodity output $q$ taken from the SUT.  We do the same for the total factor use $f$.

In [6]:
itc = Constructs.ITC(sut);

In [7]:
# Compute the total output and compare it to the total commodity output; a tolerance measure is required to account for machine precision
x = itc.L*itc.y
isapprox(x, sut.q, rtol = 1e-10)

true

In [8]:
# The corresponding factor use is calculated and compared to the empirical data; yet again a tolerance measure is required
f = itc.R*itc.L*itc.y
isapprox(f, sut.f, rtol = 1e-10)

true

Back to [table of contents](#toc)
<a id='emp_0f_lp'></a>

### 1.2 Solution by LP
Let us now solve the system via a linear program.

<a id='emp_0f_lp_rel'></a>

#### 1.2.1 Employing SUT coefficients

We start with a model in coefficient form.

In [9]:
# To set up our initial primal, we write:
primal = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(primal, g_star[1:size(V,1)] >= 0, base_name = "g*")
@objective(primal, Min, sum(sut.pii'*sut.S*g_star))
@constraint(primal, c1, (sut.C-sut.B)*(g_star) .>= sut.e);

In [10]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(c1)
@show value.(g_star);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(c1) = [51.0; 42.0; 31.0;;]
value.(g_star) = [16.925829775953638, 124.96554750130262, 122.35070431547483, 0.0, 0.0]


We can see in the output that the model found a feasible solution for both the primal and dual, with the resulting objective value being $124$, as we derived by calculating $\pi' f$. However, the modelled industry output is different from the one we calculated directly from the SUT-system earlier. The reason for this is the combined effect of by-products being present and the system being rectangular while not considering physical factor limits. It is particularly such factor constraints that enable the substitution/ complementation of technologies. In their absence, the system defaults back to a square outlay.

On the bright side, however, we can see that the shadow prices equal the unit vector; we should be reminded that $p = i$ because the given system was in monetary values.

In [11]:
@show -shadow_price.(c1);

-(shadow_price.(c1)) = [1.0; 1.0; 1.0;;]


Let us now model the dual explicitly and examine this in more detail:

In [12]:
# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@objective(dual, Max, sum(p'*sut.e))
@constraint(dual, c1d, (sut.C-sut.B)'*(p) .<= sut.S'*sut.pii);

In [13]:
# Run the model and show results
optimize!(dual)
model_solution(primal)
@show value.(p)
@show shadow_price.(c1d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(p) = [1.0, 1.0, 1.0]
shadow_price.(c1d) = [16.925829775953638; 124.96554750130262; 122.35070431547481; 0.0; 0.0;;]


While the model has a feasible solution for which the objective value is equal to that one of the primal and where the shadow prices reflect the originally observed (i.e. imported from spreadsheet) total output of the primal, we see indeed that the modelled price vector is equal the unit vector. On the other hand, however, the shadow prices for the dual, indicating the total output of the system, equal the solution vector $g*$ resulting from the primal.

Does that mean that the originally observed output vector $g$ is sub-optimal? No, as we can easily see when we calculate the dual's objective function and constraint separately with that vector:

In [14]:
println("The thus calculated objective value is: ", sut.pii'*sut.S*sut.g)
println("And the LHS of the constraint: ", (sut.C-sut.B)*sut.g)
println("And the same LHS with originally observed output levels: ", (sut.C-sut.B)*value.(g_star))

The thus calculated objective value is: [124.0;;]
And the LHS of the constraint: [51.0; 41.99999999999999; 31.000000000000014;;]
And the same LHS with originally observed output levels: [51.00000000000001, 42.0, 31.0]


With that, we see that a system consisting of more industries than commodities may have multiple alternate optimal solutions for its primal. That is, no unique solution for the output vector exists. The usefulness of the primal is hence limited in this case.

Mirroring the case of ([c>i](./SU-RCOT_c+i-.ipynb)), the reason for this behaviour is, mathematically speaking, the column-wise linear dependence of the net output (coefficients) matrix; it is thus not of full rank. Economically, according to the simplex algorithm, those market actors would produce initially which can serve the highest final demand for commodties at the lowest factor costs per unit of positive net output - at least while no physical factor limits are transgressed (here, we didn't even consider them).

In the following, we will concentrate on the primal.

<a id='emp_0f_lp_abs'></a>

#### 1.2.2 Absolute model formulation

Let us now model the primal in absolute form

In [15]:
# Instantiate the model
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

In [16]:
# To set up our initial primal, we write:
primal = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@objective(primal, Min, sum(sut.pii'*sut.F*z_star))
@constraint(primal, c1, (sut.V'-sut.U)*(z_star) .>= sut.e);

In [17]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(c1)
@show value.(z_star);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(c1) = [51.0; 42.0; 31.0;;]
value.(z_star) = [0.15818532500891247, 1.2251524264833586, 1.3903489126758504, 0.0, 0.0]


Unsurprisingly, the absolute formulation yields the same result as the coefficient-based one above. That is, the objective value is the same and also the same industries are active. Multiplying the activity level by the original industry output gives the same modelled output as above.

In [18]:
isapprox(value.(g_star), value.(z_star).*sut.g, rtol = 1e-10) 

true

<a id='emp_f_lp'></a>

## 2. Empirical system with factor constraints: solution by LP
Let us now impose factor limits that equal the observed factor use. That means that these constraints would be active. We can see that an automatic reallocation to a square system does not result anymore.

In [19]:
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [20]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.99999999999999
value.(var_con) = [0.9999999999999993, 1.0, 1.0000000000000002, 0.9999999999999996, 0.9999999999999988]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999993, 6.666666666666666, 8.148148148148147, 21.428571428571427]


Let us increase the availability of factor #f.1 to 28 and of #f.4 to 22.


In [21]:
su_rcot_data.f[1] = 28
su_rcot_data.f[4] = 22

22

In [22]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(var_con) = [1.0000000000000022, 0.9999999999999992, 0.9999999999999993, 1.000000000000003, 1.000000000000001]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.500000000000014, 6.666666666666666, 8.148148148148147, 21.42857142857142]


Although these increases in factor availability have not changed the industry activity levels, we know from earlier that the case of no binding factor constraints results in the reduction to a square system. Let us therefore now examine how much we can shift any of the variables before a change in the solution results. We do that through a sensitivity analysis as follows:

In [23]:
sensitivities(su_rcot_model,type="constraint")

Row,name,value,rhs,slack,shadow_price,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,c1,51.0,51.0,0.0,-1.0,-0.721714,0.225334
2,c1,42.0,42.0,0.0,-1.0,-0.982585,0.270116
3,c1,31.0,31.0,0.0,-1.0,-0.627836,0.206092
4,c2,27.5,28.0,0.5,0.0,-0.5,Inf
5,c2,6.66667,6.66667,0.0,0.0,-0.0744533,0.269134
6,c2,8.14815,8.14815,0.0,0.0,-0.0210169,0.0464243
7,c2,21.4286,22.0,0.571429,0.0,-0.571429,Inf


Given that the availabilities of factors #f.2 and #f.3 can change only a little (see rows 5 and 6), let us test now what the resulting solution would be if we apply such a change. We do that for factor #f.3. We see immediately that the output levels of the industries are changing non-uniformly.

In [24]:
su_rcot_data.ϕ[3] = 8.5;

In [25]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(var_con) = [0.7324539467139816, 1.0801615190470797, 1.108193933897949, 0.6420946582120927, 0.7773931168179206]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [26.39554527136912, 6.666666666666666, 8.194572471984095, 22.0]


Let us lift the endowments of each factor by a smal percentage and see which industries are then activated, simply to illustrate how sensitive the model is given the current parameterisation:

In [26]:
su_rcot_data.ϕ = su_rcot_data.ϕ*1.02;

In [27]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.00000000000001
value.(var_con) = [0.47171115341995956, 1.165576027538281, 1.2001894806730047, 0.2593186750903928, 0.6411398752171652]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [25.196863686206104, 6.8, 8.20731984316766, 22.44]


Let us lift the factor endowments yet a little further:

In [28]:
su_rcot_data.ϕ = su_rcot_data.ϕ*1.5;

In [29]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(var_con) = [0.15818532500891247, 1.2251524264833586, 1.3903489126758504, 0.0, 0.0]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [24.47900765103248, 6.1716500452480405, 8.414866139940868, 23.66052956096559]


We see that yet again a different set of industries is activated. Now that no factor constraint is active anymore, we get the same solution as before when factor constraints were not included in the model at all.

Back to [table of contents](#toc)
<a id='add_jp'></a>

## 3. Adding technology alternatives under factor constraints
So far, we have only worked with the SUT system as imported and without considering factor constraints. Let us now add new technologies to it and see how substitution may or may not occur. We use the model in its absolute form.

Back to [table of contents](#toc)
<a id='add_jp_1sp'></a>

### 3.1 Introducing an alternative single production technology
Let us now introduce a single-production alternative to #i.1 for producing #c.1 to the original SUT system, with no changed variables except the added technology.

In [30]:
# start from fresh data
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [31]:
# Define supply and use of the additional technology
V1_alt1 = [76 0 0]
U1_alt1 = [0 9.5 45.0]'
F1_alt1 = [10 0 1.3 2.8]';

In [32]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V1_alt1]
su_rcot_data.U = @views [su_rcot_data.U U1_alt1]
su_rcot_data.F = @views [su_rcot_data.F F1_alt1];

In [33]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.87797966883994
value.(var_con) = [0.0, 1.015074183045498, 0.9307046463568055, 1.2837385135683612, 0.9058591456602737, 1.1092757378188525]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999996, 6.666666666666666, 8.10381073568221, 21.427471753570455]


Given that setup, we see that the new technology substitutes the dominant one (and further complementation occurs as well.)

Back to [table of contents](#toc)
<a id='add_jp_1jp'></a>

### 3.2 Introducing an alternative joint production technology

Let us now introduce a joint-production alternative to industry #i.1 (i.e. again for producing commodity #c.1 as primary product), while also significantly increasing availability of factor #f.2 and increasing the final demand for #c.3. 

In [34]:
# start from fresh data
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [35]:
# Alter final demand and factor availability
su_rcot_data.ϕ[2] = 21
su_rcot_data.e[3] = 35;

In [36]:
# Define supply and use of the additional technology
V1_alt1 = [78 5 0]
U1_alt1 = [2 12 48]'
F1_alt1 = [10 0 1.1 2.5]';

In [37]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V1_alt1]
su_rcot_data.U = @views [su_rcot_data.U U1_alt1]
su_rcot_data.F = @views [su_rcot_data.F F1_alt1];

In [38]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 126.32314665329653
value.(var_con) = [0.0, 0.9821809977437951, 0.0, 1.2345750814740555, 5.237894209074954, 2.1498119829531213]
value.(demand_con) = [51.0, 42.0, 35.0]
value.(factor_con) = [27.499999999999996, 12.255867524581484, 6.7019648477758285, 14.812723083718321]


Yet again, another set of technologies is activated.

Let us compare this case to when we have the same technologies and parameterisation as just now, but add the same technology once more. On top of that, let us assume that #i.1 and the two alternative technologies make use of an additional factor #f.5 (priced at #pii.5).

In [39]:
# Define supply and use of the additional technology
F5 = [1 0 0 0 0 4 2]
f5 = 1
pii5 = 0.8;

In [40]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V1_alt1]
su_rcot_data.U = @views [su_rcot_data.U U1_alt1]
su_rcot_data.F = @views [su_rcot_data.F F1_alt1]

su_rcot_data.F = @views [su_rcot_data.F; F5]
su_rcot_data.ϕ = @views [su_rcot_data.ϕ; f5]
su_rcot_data.pii = @views [su_rcot_data.pii; pii5];

In [41]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 128.48413999946442
value.(var_con) = [0.1901025627292925, 1.2356064054842943, 1.0222598612859173, 0.0, 1.8143627989181361, 0.0, 0.40494871863535375]
value.(demand_con) = [51.0, 42.0, 35.0]
value.(factor_con) = [24.85572374117416, 8.63871743030822, 8.058477318868825, 21.428571428571427, 1.0]


And yet again, another set of technologies is activated.

Back to [table of contents](#toc)
<a id='add_jp_3mp_jp'></a>

---

We end here the exposition of RCOT models for the case (c<i) and suggest the reader to continue with case [(c=i)](./SU-RCOT_c=i.ipynb).